In [1]:
#引入包
import numpy as np
import pandas as pd

# 端午节的淘宝粽子交易  
## 问题  
(1) 请删除最后一列为缺失值的行，并求所有在杭州发货的商品单价均值。  

(2) 商品标题带有“嘉兴”但发货地却不在嘉兴的商品有多少条记录?  

(3) 请按照分位数将价格分为“高、较高、中、较低、低”5 个类别，再将 类别结果插入到标题一列之后，最后对类别列进行降序排序。  

(4) 付款人数一栏有缺失值吗?若有则请利用上一问的分类结果对这些缺失 值进行合理估计并填充。  

(5) 请将数据后四列合并为如下格式的 Series:商品发货地为 ××，店铺为 ××，共计 ×× 人付款，单价为 ××。  

(6) 请将上一问中的结果恢复成原来的四列。  

In [2]:
#读取数据表
df = pd.read_csv('./data/端午粽子数据.csv')
df.head()

,标题,价格,付款人数,店铺,发货地址
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,129,6人付款,五芳斋官方旗舰店,浙江 嘉兴
1,北京稻香村端午粽子手工豆沙粽220g*2袋散装豆沙粽香甜软糯豆沙粽,44,8人付款,天猫超市,上海
2,五芳斋粽子礼盒装鲜肉咸蛋黄大肉粽嘉兴豆沙甜粽端午团购散装礼品,89.9,100万+人付款,五芳斋官方旗舰店,浙江 嘉兴
3,稻香私房鲜肉粽蛋黄肉粽嘉兴粽子咸鸭蛋礼盒装端午节送礼特产团购,138,1936人付款,稻香村食品旗舰店,北京
4,嘉兴粽子 蛋黄鲜肉粽新鲜大肉粽早餐散装团购浙江特产蜜枣多口味,3.8,9500+人付款,城城喂食猫,浙江 嘉兴


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4403 entries, 0 to 4402
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   标题      4403 non-null   object
 1    价格     4403 non-null   object
 2   付款人数    4332 non-null   object
 3   店铺      4403 non-null   object
 4   发货地址    4400 non-null   object
dtypes: object(5)
memory usage: 172.1+ KB


In [4]:
#(1) 请删除最后一列为缺失值的行，并求所有在杭州发货的商品单价均值。
zongzi = df.copy().dropna(axis=0,subset=['发货地址 '])#4400条，以下问题均以此表为基础
pd.to_numeric(zongzi[zongzi['发货地址 '].str.contains('杭州')][' 价格'],errors='coerce').mean().round(decimals=2)

81.1

In [5]:
#(2) 商品标题带有“嘉兴”但发货地却不在嘉兴的商品有多少条记录?
zongzi[(zongzi['标题'].str.contains('嘉兴')) &(~zongzi['发货地址 '].str.contains('嘉兴'))].shape[0]

1032

In [6]:
#(3) 请按照分位数将价格分为“高、较高、中、较低、低”5 个类别，再将 类别结果插入到标题一列之后，最后对类别列进行降序排序。
zongzi.loc[:,' 价格'] = pd.to_numeric(zongzi.loc[:,' 价格'],errors='coerce')
q = [zongzi.loc[:,' 价格'].quantile(i) for i in [0,.2,.4,.6,.8,1]];q[-1] += 1
zongzi.loc[:,'价格类别'] = pd.cut(zongzi[' 价格'],q,right=False,labels=['低','较低','中','较高','高'])
zongzi.loc[:,['标题','价格类别', ' 价格', '付款人数', '店铺', '发货地址 ']].sort_values(by='价格类别')

,标题,价格类别,价格,付款人数,店铺,发货地址
3666,嘉兴五芳斋粽子紫米栗蓉粽板栗栗子甜粽子五谷素粽真空袋装美味,低,11.0,162人付款,三珍斋食品专营店,江苏 南京
1623,白米粽子新鲜湖南甜粽原味小粽子农家手工散装绿豆红枣碱水清水粽,低,20.9,181人付款,乐家_stone,湖南 常德
1053,【9只特价】新鲜蛋黄肉粽 嘉兴粽子特产端午节批发团购早餐粽手工,低,20.8,1358人付款,怡品三清旗舰店,浙江 杭州
3743,三珍斋嘉兴粽子大肉粽子蛋黄肉粽豆沙蜜枣端午节散装批发鲜肉棕子,低,19.9,35人付款,三珍斋乌镇专卖,浙江 嘉兴
1311,现做新鲜粽子咸蛋黄肉粽甜粽端午节散装嘉兴蜜枣豆沙大批發礼盒装,低,9.9,5000+人付款,徽老九美食,安徽 亳州
...,...,...,...,...,...,...
3548,锦华粽子瑞阳祥礼端午节礼盒新鲜嘉兴肉素六粒粽蛋卷组合送人礼品,高,110.6,36人付款,北京咏逢礼信商贸有限公司,北京
3523,印象昆韵新品昆曲文化创意端午节胚芽糙米手工肉粽子端阳礼品礼盒,高,156.0,21人付款,印象昆韵,江苏 苏州
0,五芳斋粽子礼盒 心悦+18只装咸鸭蛋组合端午节礼品团购嘉兴肉粽子,高,129.0,6人付款,五芳斋官方旗舰店,浙江 嘉兴
538,五芳斋粽子蛋黄鲜肉粽豆沙粽栗子肉棕子新鲜散装批发团购嘉兴肉粽,NaN,NaN,1.0万+人付款,五芳斋官方旗舰店,浙江 嘉兴


In [7]:
#(4) 付款人数一栏有缺失值吗?若有则请利用上一问的分类结果对这些缺失 值进行合理估计并填充。
zongzi_loss = zongzi.copy()
zongzi_loss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4400 entries, 0 to 4402
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   标题      4400 non-null   object  
 1    价格     4398 non-null   float64 
 2   付款人数    4329 non-null   object  
 3   店铺      4400 non-null   object  
 4   发货地址    4400 non-null   object  
 5   价格类别    4398 non-null   category
dtypes: category(1), float64(1), object(4)
memory usage: 210.7+ KB


In [8]:
#用同价格类别的付款人数均值（不考虑100余条类似1.0万+、5000+等较大数值。）
grouped = zongzi_loss.groupby('价格类别')
for name,group in grouped:
    zongzi_loss.loc[group.index,'付款人数'] = str(int(pd.to_numeric(group['付款人数'].astype(str).str[:-3],errors='coerce').mean()))+'人付款'
zongzi_loss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4400 entries, 0 to 4402
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   标题      4400 non-null   object  
 1    价格     4398 non-null   float64 
 2   付款人数    4400 non-null   object  
 3   店铺      4400 non-null   object  
 4   发货地址    4400 non-null   object  
 5   价格类别    4398 non-null   category
dtypes: category(1), float64(1), object(4)
memory usage: 370.7+ KB


In [9]:
#(5) 请将数据后四列合并为如下格式的 Series:商品发货地为 ××，店铺为 ××，共计 ×× 人付款，单价为 ××。
df_series = zongzi_loss.astype(str)
df_str = ('商品发货地为'+df_series['发货地址 ']+'，店铺为'+df_series['店铺']+'，共计'+df_series['付款人数']+'，单价为'+df_series[' 价格']+'。')
df_str.head()

0    商品发货地为浙江 嘉兴，店铺为五芳斋官方旗舰店，共计139人付款，单价为129.0。
1            商品发货地为上海，店铺为天猫超市，共计357人付款，单价为44.0。
2     商品发货地为浙江 嘉兴，店铺为五芳斋官方旗舰店，共计291人付款，单价为89.9。
3       商品发货地为北京，店铺为稻香村食品旗舰店，共计139人付款，单价为138.0。
4         商品发货地为浙江 嘉兴，店铺为城城喂食猫，共计540人付款，单价为3.8。
dtype: object

In [10]:
#(6) 请将上一问中的结果恢复成原来的四列。
df_res = df_str.str.extract(r'商品发货地为(?P<发货地址>[\w]+\s*[\w]+)，店铺为(?P<店铺>[\w]+)，共计(?P<付款人数>[\w]+\+*[\w]+)，单价为(?P<价格>[\d]+\.*[\d]+)。')
df_res.head()

,发货地址,店铺,付款人数,价格
0,浙江 嘉兴,五芳斋官方旗舰店,139人付款,129.0
1,上海,天猫超市,357人付款,44.0
2,浙江 嘉兴,五芳斋官方旗舰店,291人付款,89.9
3,北京,稻香村食品旗舰店,139人付款,138.0
4,浙江 嘉兴,城城喂食猫,540人付款,3.8
